### Load libraries

In [1]:
import sys
sys.path.insert(0, '../scripts/')
from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model
from load_modify_sample_utils import sample_dingo  #, sample_optgp
from correlations_utils import correlated_reactions, plot_correlation_matrix, split_forward_reverse
from correlations_utils import find_reactants_products, sharing_metabolites_square_matrix

### Load model

In [2]:
ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)

Set parameter Username
Set parameter LicenseID to value 2634947
Academic license - for non-commercial use only - expires 2026-03-11
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

### Modify model and sample

In [3]:
ec_cobra_model_condition_1, ec_dingo_model_condition_1 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=100)
ec_cobra_model_condition_2, ec_dingo_model_condition_2 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=0)


samples_dingo_condition_1 = sample_dingo(ec_dingo_model_condition_1, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_1.shape)

samples_dingo_condition_2 = sample_dingo(ec_dingo_model_condition_2, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_2.shape)



Read LP format model from file /tmp/tmpiwxuyhji.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp0ziytdn4.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Set parameter Username
Set parameter LicenseID to value 2634947
Academic license - for non-commercial use only - expires 2026-03-11
phase 1: number of correlated samples = 500, effective sample size = 27, ratio of the maximum singilar value over the minimum singular value = 2770.47
phase 2: number of correlated samples = 500, effective sample size = 5, ratio of the maximum singilar value over the minimum singular value = 295.578
phase 3: number of correlated samples = 500, effective sample size = 11, ratio of the maximum singilar value over the minimum singular value = 308.246
phase 4: number of correlated samples = 500, effective sample size = 126, ratio of the maximum singilar value over the minimum singular value = 24.2281
phase 5: number of correlated sa

[5]maximum marginal PSRF: 1.01286


phase 1: number of correlated samples = 500, effective sample size = 3, ratio of the maximum singilar value over the minimum singular value = 2566.2
phase 2: number of correlated samples = 500, effective sample size = 6, ratio of the maximum singilar value over the minimum singular value = 73.0992
phase 3: number of correlated samples = 500, effective sample size = 3, ratio of the maximum singilar value over the minimum singular value = 518.497
phase 4: number of correlated samples = 500, effective sample size = 80, ratio of the maximum singilar value over the minimum singular value = 63.8079
phase 5: number of correlated samples = 500, effective sample size = 197, ratio of the maximum singilar value over the minimum singular value = 3.2389
phase 6: number of correlated samples = 500, effective sample size = 182, ratio of the maximum singilar value over the minimum singular value = 2.64119
phase 7: number of correlated samples = 2400, effective sample size = 1107
phase 8: number of cor

[5]maximum marginal PSRF: 1.01424


### Split reactions having both positive and negative flux values (reversible reactions) into two separate reactions (forward and reverse).

In [4]:

extended_steady_states, extended_reactions = split_forward_reverse(samples_dingo_condition_2, reactions=ec_dingo_reactions)


reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions = find_reactants_products(ec_cobra_model, extended_reactions)
print(len(reversibility_list_all_reactions), len(reactants_list_all_reactions))



118 118


### Calculate a boolean matrix with dimensions equal to the number of reactions (here, reactions after the forward and reverse split). "True" in a position of the 2D array represents 2 reactions with a sharing reactant or product. This is done to filter correlations between reactions that do not share metabolites

In [5]:

boolean_sharing_metabolites_matrix = sharing_metabolites_square_matrix(extended_reactions, reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions)


print(boolean_sharing_metabolites_matrix)
print(boolean_sharing_metabolites_matrix.shape)


[[ True False  True ... False False False]
 [False  True False ... False False False]
 [ True False  True ... False False False]
 ...
 [False False False ...  True False False]
 [False False False ... False  True False]
 [False False False ... False False  True]]
(118, 118)


### Compute pairwise linear and non-linear correlations from flux samples and filter (remove) correlations between reactions not sharing any metabolites

In [6]:
(linear_correlation_matrix, 
non_linear_correlation_matrix, 
mixed_correlation_matrix, 
correlations_dictionary) = correlated_reactions(
        steady_states = extended_steady_states,
        boolean_sharing_metabolites_matrix=boolean_sharing_metabolites_matrix,
        reactions=extended_reactions, 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True
)

/opt/miniconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/miniconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


### Show some pairs of reactions with extreme non-linear dependencies

note: the following seems to be empty..?

In [7]:

filtered_positive = {
    pair: metrics
    for pair, metrics in correlations_dictionary.items()
    if metrics['jensenshannon'] > 0.10 and metrics['indicator'] > 1.2
}

print(filtered_positive)
print(filtered_positive.keys())


filtered_negative = {
    pair: metrics
    for pair, metrics in correlations_dictionary.items()
    if metrics['jensenshannon'] < -0.10 and metrics['indicator'] < 1.2

}

print(filtered_negative)
print(filtered_negative.keys())


{}
dict_keys([])
{}
dict_keys([])


### Plot 3 correlation matrices, (a) only with linear correlations, (b) only with non-linear correlations and (c) with both linear and non-linear correlations

In [8]:

cobra_reactions_str = [str(reaction.id) for reaction in ec_cobra_model.reactions]

plot_correlation_matrix(linear_correlation_matrix, extended_reactions)
plot_correlation_matrix(non_linear_correlation_matrix, extended_reactions)
plot_correlation_matrix(mixed_correlation_matrix, extended_reactions)


### Calculate pairwise correlatins without filtering for sharing metabolites

In [9]:

(linear_correlation_matrix_no_metabolites, 
non_linear_correlation_matrix_no_metabolites, 
mixed_correlation_matrix_no_metabolites, 
correlations_dictionary_no_metabolites) = correlated_reactions(
        steady_states = extended_steady_states,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions, 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)


/opt/miniconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning:

invalid value encountered in divide

/opt/miniconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning:

invalid value encountered in divide



In [10]:

filtered_g = {
    pair: metrics
    for pair, metrics in correlations_dictionary_no_metabolites.items()
    if metrics['jensenshannon'] > 0.1 and metrics['indicator'] > 1.2
}

print(filtered_g)
print(filtered_g.keys())


filtered_l = {
    pair: metrics
    for pair, metrics in correlations_dictionary_no_metabolites.items()
    if metrics['jensenshannon'] < -0.1 and metrics['indicator'] < 1.2

}

print(filtered_l)
print(filtered_l.keys())


{}
dict_keys([])
{}
dict_keys([])


In [11]:

cobra_reactions_str = [str(reaction.id) for reaction in ec_cobra_model.reactions]

plot_correlation_matrix(linear_correlation_matrix_no_metabolites, extended_reactions)
plot_correlation_matrix(non_linear_correlation_matrix_no_metabolites, extended_reactions)
plot_correlation_matrix(mixed_correlation_matrix_no_metabolites, extended_reactions)
